# Exploratory Data Analysis For Crop Yeild Predictions
## Project Objectives:
1. Understand the dataset and the problem
2. Import and inspect the data
3. Visualize the data
4. Preprocess data (cleaning and handling of missing values/outliers)
5. Data transformation
6. Attribute relationships and correlation
7. Model Development
8. Model Evaluation and Refinement


## Assumptions:
1. Location is USA and region describes where in the USA the sample was gathered
2. Weather condition is the most frequent type of weather the sample incurred during its growth cycle

The problem here is **Crop Yeild Prediction** using a variety of variables such as climate data, location data, crop type, and farming practices. The dataset is of CSV format and includes categorical, bool, and numeric attributes.

## Step 1: Import Necessary Libraries 

In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
import os
import gc
from scipy import stats
import warnings
%matplotlib inline
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

## Step 2: Import and Inspect Dataset

In [2]:
filepath = 'crop_yield.csv'

In [ ]:
headers = ['region','soil_type','crop','rainfall_mm','temp_c','fertilizer_use','irrigation_use','weather','days_to_harvest','yield_per_hect']
data = pd.read_csv(filepath)
data.columns = headers
data

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
null_values = data.isna().sum()
print("Null Values Per Column")
print(null_values, "\n")

duplicated_rows = data.duplicated().sum()
print("Number of Duplicated Rows in Dataset")
print(duplicated_rows)

### Using the information gathered from initial inspection we can see that this is a complete dataset with no null values. It can be seen that the minimum value from crop yeild  is negative, this should not be the case and will be treated as an error. 

## Removal of rows containing a negative yield

In [7]:
data = data[data['yield_per_hect'] >=0]

In [8]:
data.shape

(999769, 10)

In [ ]:
data.info()

## Step 3: Understand and Visualize the Data


### First we will plot the distributions of our contiunuos variables using histograms

In [10]:
sns.distplot(data['yield_per_hect']).set_title("Crop Yeild Distrubution")

Text(0.5, 1.0, 'Crop Yeild Distrubution')

In [11]:
sns.distplot(data['rainfall_mm']).set_title("Distribution of Rainfall")

Text(0.5, 1.0, 'Distribution of Rainfall')

In [12]:
sns.distplot(data['temp_c'])

<Axes: title={'center': 'Distribution of Rainfall'}, xlabel='temp_c', ylabel='Density'>

### Now we will plot the counts of our categorical variables using count plots

In [13]:
sns.countplot(data=data, x='soil_type').set_title("Count of Soil Types")

Text(0.5, 1.0, 'Count of Soil Types')

In [14]:
sns.countplot(data=data, x='crop').set_title("Count of Crop Types")

Text(0.5, 1.0, 'Count of Crop Types')

In [15]:
sns.countplot(data=data, x='weather').set_title("Count of Weather Conditions")

Text(0.5, 1.0, 'Count of Weather Conditions')

In [16]:
sns.countplot(data=data, x='region').set_title("Count of Regions")

Text(0.5, 1.0, 'Count of Regions')

In [ ]:
sns.regplot(x='rainfall_mm',y='yield_per_hect', data=data).set_title("Crop Yield Rainfall Distribution")

In [ ]:
sns.regplot(x='temp_c',y='yield_per_hect', data=data).set_title("Crop Yield Temperature Distribution")

## Step 4: Attribute relationships and correlation

In [ ]:
data['color'] = data['irrigation_use'].map({True: 'blue', False: 'red'})

plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, y='crop', x='yield_per_hect', hue='color', palette={'blue': 'blue', 'red': 'red'}, s=50)

plt.title('Yield of Crop by Crop Type and Irrigation Usage')
plt.xlabel('Yield of Crop (per hectare)')
plt.ylabel('Crop Type')

handles = [
    plt.Line2D([0], [0], marker='o', color='w', label='Used', markerfacecolor='blue', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Not Used', markerfacecolor='red', markersize=10)
]
plt.legend(title='Irrigation Use', handles=handles, loc='upper right')


plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
scatter = sns.scatterplot(data=data, x='yield_per_hect', y='temp_c', hue='crop', palette='Set1', s=50)

plt.title('Yield vs Temperature by Crop Type')
plt.xlabel('Crop Yield (per hectare)')
plt.ylabel('Temperature (°C)')
plt.legend(title='Crop Type', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, y='crop', x='yield_per_hect', hue='soil_type', palette='Set1', s=50)

plt.title('Yield of Crop by Soil Type')
plt.xlabel('Yield of Crop (per hectare)')
plt.ylabel('Crop Type')

plt.legend(title='Soil Type', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, y='crop', x='yield_per_hect', hue='weather', palette='Set1', s=50)

plt.title('Yield of Crop by Weather Conditions')
plt.xlabel('Yield of Crop (per hectare)')
plt.ylabel('Crop Type')

plt.legend(title='Weather Condition', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In [ ]:
data['color_fert'] = data['fertilizer_use'].map({True: 'green', False: 'red'})

plt.figure(figsize=(10, 6))
sns.scatterplot(data=data, y='crop', x='yield_per_hect', hue='color_fert', palette={'green': 'green', 'red': 'red'}, s=50)

plt.title('Yield of Crop by Crop Type and Fertilizer Usage')
plt.xlabel('Yield of Crop (per hectare)')
plt.ylabel('Crop Type')

handles = [
    plt.Line2D([0], [0], marker='o', color='w', label='Used', markerfacecolor='green', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Not Used', markerfacecolor='red', markersize=10)
]
plt.legend(title='Fertilizer Use', handles=handles, loc='upper right')

plt.show()

In [ ]:
cat_cols = ['soil_type','crop','fertilizer_use','irrigation_use','weather']
for cols in cat_cols:
    yield_summary = data.groupby(cols)['yield_per_hect'].agg(['sum','mean']).reset_index()
    yield_summary.columns = [cols, 'total_yield', 'average_yield']
    print(yield_summary,'\n')
    

## step 5: Data Trasformation - One-Hot Encoding for categorical Variables

In [ ]:
data_numeric = pd.get_dummies(data=data, columns=['region','soil_type','crop','weather'])

for col in data_numeric.columns:
    if data_numeric[col].dtype == 'bool':
        data_numeric[col] = data_numeric[col].astype(int)

data_numeric.info()

In [ ]:
correlation = data_numeric.corr()['yield_per_hect'].sort_values(ascending=False)
print(correlation)

## Step 5: Preprocess The Data

### Inital step will be to round our float variables down

In [ ]:
data_numeric['yield_per_hect'] = data_numeric['yield_per_hect'].round(4)
data_numeric['rainfall_mm'] = data_numeric['rainfall_mm'].round(2)
data_numeric['temp_c'] = data_numeric['temp_c'].round(2)
data_numeric.shape

In [ ]:
data.shape

### Now we will remove unecessary columns

In [ ]:
data_numeric = data_numeric.drop('days_to_harvest', axis=1)

In [ ]:
data_numeric = data_numeric.reset_index(drop=True)

## Step 6: Model Creation

### Our inital model will be a standard **Multiple Linear Regression Model**

In [ ]:
x_data = data_numeric.drop('yield_per_hect', axis=1)
y_data = data_numeric['yield_per_hect']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, random_state=1)
print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

In [ ]:
lm = LinearRegression()
lm.fit(x_train,y_train)
Y_hat = lm.predict(x_test)

In [ ]:
lm.score(x_test,y_test)

In [ ]:
plt.figure(figsize=(10, 6))
ax1 = sns.distplot(data_numeric['yield_per_hect'], hist=False, color="r", label="Actual Value")
sns.distplot(Y_hat, hist=False, color="b", label="Fitted Values" , ax=ax1)

plt.title('Actual vs Fitted Values for Yield')
plt.xlabel('Yield Per Hectare')
plt.ylabel('Density')
plt.show()

In [ ]:
rcross = cross_val_score(lm,x_data, y_data, cv=4)
mse = mean_squared_error(y_test, Y_hat)

In [ ]:
print("Cross Validation Scores:", rcross)
print("Cross Validation Mean:",rcross.mean())

print("Mean-Squared_error:",mse)

## Conclusions from initial Model Construction:
- Model performs decent but shows signs of over fitting and lack of generalization.
- Model visualizations show "steps" and do not smoothen to the actual distribution
- R2 Score and cross validation score are consistent showing good progress

With our current linear regression model we acheive a R2 score of 0.913, and when using a cross validation score with 4 folds we get a mean of 0.912. This shows consistensy and decent performance. We will now refine this model by adding polynomial features, using regularization, and performing a grid search for hyperparameters.

## Next Steps: Implement Polynolinomial features to attributes not displaying a linear relationship with target variable (Yield)

### Attributes with Linear Relationship: Rainfall mm, Fertilizer Use, Irrigation Use
### Attributes without a Linear Relationship: Region, Soil Type, Crop, Temperature, Weather

In [ ]:
data_numeric.columns

In [ ]:
data_numeric.info()

In [ ]:
linear_columns = ['rainfall_mm', 'fertilizer_use', 'irrigation_use']

non_linear_columns = ['region_East', 'region_North', 'region_South',
       'region_West', 'soil_type_Chalky', 'soil_type_Clay', 'soil_type_Loam',
       'soil_type_Peaty', 'soil_type_Sandy', 'soil_type_Silt', 'crop_Barley',
       'crop_Cotton', 'crop_Maize', 'crop_Rice', 'crop_Soybean', 'crop_Wheat',
       'weather_Cloudy', 'weather_Rainy', 'weather_Sunny']

target_column = 'yield_per_hect'

In [ ]:
linear_features = data_numeric[linear_columns]
non_linear_features = data_numeric[non_linear_columns]

In [ ]:
poly = PolynomialFeatures(degree=2, include_bias=False)
non_linear_poly = poly.fit_transform(non_linear_features)

In [ ]:
non_linear_poly_df = pd.DataFrame(non_linear_poly, columns=poly.get_feature_names_out(non_linear_columns))

In [ ]:
final_features = pd.concat([linear_features, non_linear_poly_df], axis=1)

In [ ]:
X = final_features 
y = data_numeric[target_column]  

In [ ]:
print(f"Number of rows in X (final_features): {len(final_features)}")
print(f"Number of rows in y (target_column): {len(data_numeric[target_column])}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

In [ ]:
plt.scatter(y_test, y_pred)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  
plt.xlabel('Actual Yield')
plt.ylabel('Predicted Yield')
plt.title('Actual vs Predicted Yield')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

ax1 = sns.kdeplot(data_numeric['yield_per_hect'], color="r", label="Actual Value", fill=False)

sns.kdeplot(y_pred, color="b", label="Fitted Values", fill=False, ax=ax1)

plt.title('Actual vs Fitted Values for Yield')
plt.xlabel('Yield Per Hectare')
plt.ylabel('Density')

plt.legend()
plt.show()

## Conclusions From Polynomial Feature Implementation:
- Did not improve the model
- Lowered r2 score, increased MSE

## Next Steps: Introduce Ridge Regression and Gridsearching Hyperparameters.

In [ ]:
RidgeModel = Ridge(alpha=1)

In [ ]:
RidgeModel.fit(X_train,y_train)

In [ ]:
yhat=RidgeModel.predict(X_test)

In [ ]:
print(r2_score(y_test,yhat))

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(y_test, yhat, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2)  # Line for perfect predictions
plt.xlabel('Actual Yield')
plt.ylabel('Predicted Yield')
plt.title('Actual vs Predicted Yield (Ridge Regression)')
plt.legend()
plt.show()

In [ ]:
selected_features = ['rainfall_mm', 'fertilizer_use', 'irrigation_use', 'temp_c']
X = data_numeric[selected_features]
y = data_numeric['yield_per_hect']

poly = PolynomialFeatures(degree=2, include_bias=False)  
X_poly = poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

model = Ridge(alpha=0.1) 
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R2 Score: {r2}")

plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', linewidth=2)  
plt.xlabel('Actual Yield')
plt.ylabel('Predicted Yield')
plt.title('Actual vs Predicted Yield (Polynomial Model with Selected Features)')
plt.show()

plt.figure(figsize=(10, 6))
ax = sns.kdeplot(y_test, color="r", label="Actual Value", fill=False)
sns.kdeplot(y_pred, color="b", label="Fitted Values", fill=False, ax=ax)
plt.title('Actual vs Fitted Values for Yield (Polynomial Model)')
plt.xlabel('Yield Per Hectare')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
coefficients = model.coef_

feature_names = poly.get_feature_names_out(selected_features)

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

coef_df['abs_coefficient'] = coef_df['Coefficient'].apply(np.abs)
coef_df = coef_df.sort_values(by='abs_coefficient', ascending=False)

plt.figure(figsize=(10, 8))
sns.heatmap(coef_df[['Coefficient']].T, cmap='coolwarm', annot=True, cbar=True, center=0, fmt='.2f', 
            linewidths=1, xticklabels=coef_df['Feature'], yticklabels=False)
plt.title('Feature Importance Heatmap')
plt.show()